In [ ]:
# Install dependencies
!pip install -q numerapi pandas pyarrow matplotlib lightgbm scikit-learn cloudpickle scipy==1.10.1



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import json

# Initialize NumerAPI - the official Python API client for Numerai
from numerapi import NumerAPI
import pandas as pd

napi = NumerAPI()

# Set data version to one of the latest datasets
DATA_VERSION = "v4.3"

# read the metadata and display
feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
for metadata in feature_metaadata:
  print(metadata, len(feature_metadata[metadata]))

# Define our feature set
feature_sets = feature_metadata["feature_sets"]
for feature_set in ["small", "medium", "all"]:
  print(feature_set, len(feature_sets[feature_set]))

# download the feature metadata file
#napi.download_dataset(f"{DATA_VERSION}/features.json");

feature_set = feature_sets["medium"]


feature_stats 2376
feature_sets 17
targets 41
small 42
medium 705
all 2376


In [ ]:

# Download the training data - this will take a few minutes
#napi.download_dataset(f"{DATA_VERSION}/train_int8.parquet");

# Load only the "medium" feature set to
# Use the "all" feature set to use all features
train = pd.read_parquet(
    f"{DATA_VERSION}/train_int8.parquet",
    columns=["era", "target"] + feature_set
)
a
# Downsample to every 4th era to reduce memory usage and speedup model training (suggested for Colab free tier)
# Comment out the line below to use all the data
train = train[train["era"].isin(train["era"].unique()[::4])]

In [ ]:
train

,era,target,feature_abating_unadaptable_weakfish,feature_ablest_mauritanian_elding,feature_acclimatisable_unfeigned_maghreb,feature_accommodable_crinite_cleft,feature_accretive_sorrier_skedaddle,feature_acetose_periotic_coronation,feature_adam_incantational_winemaker,feature_additive_untrustworthy_hierologist,...,feature_witchy_orange_muley,feature_wombed_liberatory_malva,feature_won_stalwart_eisenstein,feature_wrathful_prolix_colotomy,feature_wrinkliest_unmaintainable_usk,feature_wrought_muckier_temporality,feature_yauld_antediluvial_subprefecture,feature_yelled_hysteretic_eath,feature_yoruban_unapplied_tawse,feature_zygodactyl_exponible_lathi
id,,,,,,,,,,,,,,,,,,,,,
n003bba8a98662e4,0001,0.25,0,4,0,4,2,0,2,1,...,0,0,2,4,2,4,3,2,2,3
n003bee128c2fcfc,0001,0.75,4,2,2,2,2,3,2,1,...,3,3,2,0,2,2,1,3,2,1
n0048ac83aff7194,0001,0.25,4,4,2,0,2,0,2,4,...,0,1,2,0,2,3,2,1,2,2
n00691bec80d3e02,0001,0.75,1,4,1,1,2,0,2,2,...,2,1,2,2,2,3,2,2,2,2
n00b8720a2fdc4f2,0001,0.50,0,2,0,0,2,0,2,3,...,1,1,2,0,2,0,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffc2d5e4b79a7ae,0573,0.25,4,2,4,4,3,1,0,0,...,4,1,2,1,3,2,2,2,1,1
nffc7d24176548a4,0573,0.50,0,3,3,4,2,0,2,3,...,2,0,2,4,2,0,2,1,2,3
nffc9844c1c7a6a9,0573,0.50,4,1,1,2,0,3,2,4,...,1,3,0,3,0,2,0,0,1,2


In [ ]:
import lightgbm as lgb


model = lgb.LGBMRegressor(
  n_estimators = 20000,
  learning_rate = 0.001,
  max_depth = 6,
  num_leaves = 2**6,
  colsample_bytree = 0.1,
  num_threads = 8
  )

# This will take a few minutes
model.fit(
  train[feature_set],
  train["target"]
);

In [ ]:
# Load the validation data and filter for data_type == "validation"
validation = pd.read_parquet(
    f"{DATA_VERSION}/validation_int8.parquet",
    columns=["era", "data_type", "target"] + feature_set
)
validation = validation[validation["data_type"] == "validation"]
del validation["data_type"]

# Downsample to every 4th era to reduce memory usage and speedup evaluation (suggested for Colab free tier)
# Comment out the line below to use all the data (slower and higher memory usage, but more accurate evaluation)
validation = validation[validation["era"].isin(validation["era"].unique()[::4])]

# Eras are 1 week apart, but targets look 20 days (o 4 weeks/eras) into the future,
# so we need to "embargo" the first 4 eras following our last train era to avoid "data leakage"
last_train_era = int(train["era"].unique()[-1])
eras_to_embargo = [str(era).zfill(4) for era in [last_train_era + i for i in range(4)]]
validation = validation[~validation["era"].isin(eras_to_embargo)]

# Generate predictions against the out-of-sample validation features
# This will take a few minutes
validation["prediction"] = model.predict(validation[feature_set])
validation[["era", "prediction", "target"]]

,era,prediction,target
id,,,
n002a15bc5575bbb,0579,0.502701,0.25
n00309caaa0f955e,0579,0.512892,0.50
n00576b397182463,0579,0.496799,0.50
n00633405d59c6a1,0579,0.502913,1.00
n008c2eefc8911c7,0579,0.493118,0.50
...,...,...,...
nffdb6e40e2f845c,1103,0.516022,0.25
nffe7ab06ef8a5a5,1103,0.513613,0.50
nffeddeb986175f9,1103,0.486186,0.50


In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(validation["target"], validation['prediction'])

0.049780553167508075